In [1]:
# cd into project directory 
%cd /home/baldeeb/Documents/Projects/ContrastiveLearningOfDenseDescriptor

%load_ext autoreload
%autoreload 2

import numpy as np
import torch

from tqdm import tqdm 


/home/baldeeb/Documents/Projects/ContrastiveLearningOfDenseDescriptor


In [2]:
# Tools needed to setup data loader
from dataset import make_data_loader

class dataset_cfg():
    dataset = 'unreal_parts'
    data_dir = '../simple_data'
    image_type = 'RGB'
    batch_size = 1
    workers = 1

# Mean std stuff 

In [ ]:
# Calculate dataset Mean and Standard dev
unreal_data = make_data_loader(split='train', args=dataset_cfg(), return_dataset=True)
unreal_data.iterate_randomly = False

image_count = len(unreal_data)
total_pixel_values = torch.zeros(3)
for i, data in enumerate(tqdm(unreal_data)):
    if i >= image_count: break
    total_pixel_values += data['image'].sum(dim=[1,2])

pixel_count_per_image = unreal_data.width * unreal_data.height
total_pixel_count = pixel_count_per_image * image_count
dataset_mean = total_pixel_values / total_pixel_count 

total_mean_dist_squared = torch.zeros(3)
for i, data in enumerate(tqdm(unreal_data)):
    if i >= image_count: break
    temp = data['image'].reshape(3, -1) - dataset_mean.unsqueeze(1)
    total_mean_dist_squared += temp.square().sum(dim=1)
dataset_std = (total_mean_dist_squared/count).sqrt()

dataset_mean, dataset_std

In [ ]:

# ## Approximating sample mean and variance
# # Calculate dataset Mean and Standard dev
# unreal_data = make_data_loader(split='train', args=dataset_cfg(), return_dataset=True)
# unreal_data.iterate_randomly = False

# image_count = len(unreal_data)
# total_pixel_values = torch.zeros(3)
# total_pixel_squared_values = torch.zeros(3)
# for i, data in enumerate(tqdm(unreal_data)):
#     if i >= image_count: break
#     total_pixel_values += data['image'].sum(dim=[1,2])
#     total_pixel_squared_values += data['image'].square().sum(dim=[1,2])

# pixel_count_per_image = unreal_data.width * unreal_data.height
# total_pixel_count = pixel_count_per_image * image_count

# dataset_mean = total_pixel_values / total_pixel_count
# dataset_std = (total_pixel_squared_values / total_pixel_count) - dataset_mean.square()

# dataset_mean, dataset_std

In [ ]:
dataset_mean, dataset_std = [0.5183, 0.5747, 0.7210], [0.3218, 0.3045, 0.2688]
unreal_data = make_data_loader(split='train', args=dataset_cfg(), return_dataset=True)
image = None
for data in unreal_data: image = data['image']; break

from torchvision.transforms import Normalize
N = Normalize(mean=dataset_mean, std=dataset_std)
normalized = N(image)

print(f"image shape: {normalized.shape}")
for i in range(3):
    print(f"channel {i}, mean: {normalized[i].mean()},  standard dev: {normalized[i].std()}")